In [2]:
# Prerequisets
!pip install pynetlogo
!pip install JPype1

     |████████████████████████████████| 48 kB 182 kB/s 
  Created wheel for pynetlogo: filename=pyNetLogo-0.4.1-py3-none-any.whl size=64516 sha256=bf8a40c5e8446cbca9adf286f1779b182bc7476ef5778e9660547ac4000fd1dc
  Stored in directory: /Users/ricoherzog/Library/Caches/pip/wheels/73/a0/47/93a2adf66bc7381ec21a1be6be2f774a49e4a5ee7a9ef21e54
Successfully built pynetlogo


In [1]:
# import all necessary python libraries and the pyNetLogo library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
sns.set_context('talk')
import pyNetLogo

from SALib.sample import latin
from SALib.analyze import sobol
from SALib.sample import saltelli

In [2]:
# set up the model
netlogo = pyNetLogo.NetLogoLink(gui=False) #Mac users should set this to False
netlogo.load_model('Model.nlogo')

# Setup Experiments
Specify the number of samples and the number of repetitions for each sample below.

In [3]:
# Parameters
no_samples = 100
no_repetitions = 100

### Experiment Sampling
Perform Latin Hypercube Sampling from all possible levers to ensure an evenly distributed sample of the lever space.

In [4]:
problem = {
  'num_vars': 7,
  'names': ['administrative-network-meetings',
           'total-project-proposal-frequency',
           'informal-meetings-frequency',
           'green-energy-openness-change',
           'political-variety-change',
           'max-project-capacity',
           'random-intial-trust'],
  'bounds': [[0, 25],
            [0, 25],
            [0, 25],
            [-5,5],
            [-5,5],
            [0, 25],
            [0, 1]]
}

param_values = np.round(latin.sample(problem, no_samples))
df = pd.DataFrame(param_values)
df.iloc[:,-1] = df.iloc[:,-1].astype("bool")
param_values = np.array(df)
param_values

array([[18.0, 2.0, 21.0, ..., -3.0, 13.0, True],
       [23.0, 24.0, 23.0, ..., -3.0, 10.0, False],
       [9.0, 6.0, 7.0, ..., 4.0, 17.0, True],
       ...,
       [5.0, 14.0, 4.0, ..., -3.0, 16.0, True],
       [1.0, 13.0, 11.0, ..., -1.0, 1.0, True],
       [2.0, 22.0, 9.0, ..., 2.0, 24.0, False]], dtype=object)

In [5]:
# Create a dataframe that can later be saved
param_values_df = pd.DataFrame(param_values, columns=['administrative-network-meetings',         'total-project-proposal-frequency', 'informal-meetings-frequency', 'green-energy-openness-change',        'political-variety-change', 'max-project-capacity', 'random-intial-trust'])
param_values_df

,administrative-network-meetings,total-project-proposal-frequency,informal-meetings-frequency,green-energy-openness-change,political-variety-change,max-project-capacity,random-intial-trust
0,18,2,21,-1,-3,13,True
1,23,24,23,5,-3,10,False
2,9,6,7,-0,4,17,True
3,19,6,8,-0,-4,21,False
4,8,0,17,5,-1,4,True
...,...,...,...,...,...,...,...
295,4,23,2,-2,-2,19,False
296,11,21,13,-4,4,11,False
297,5,14,4,-2,-3,16,True
298,1,13,11,3,-1,1,True


### Run Samples in NetLogo

In [6]:
idx = pd.MultiIndex.from_product([[],[]],names=['Run', 'Random Seed'])
col = ['MW_implemented_wind', 'MW_implemented_solar', 'MW_implemented_urban']
results = pd.DataFrame('', idx, col)

for run in range(param_values.shape[0]):
    for rs in range(no_repetitions):
        netlogo.command('setup')
        netlogo.command('random-seed {}'.format(rs))
        
        # Set the input parameters
        for i, name in enumerate(problem['names']):
            # Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, param_values[run,i]))


        # Run for 100 ticks and return the number of sheep and wolf agents at 
        # each time step
        MW_implemented = netlogo.repeat_report(['current-wind-production','current-solar-production', 'current-urban-production'], 360)
    
        # For each run, save the value of MW_implemented
        results.loc[(run, rs), 'MW_implemented_wind'] = MW_implemented['current-wind-production'].max()
        results.loc[(run, rs), 'MW_implemented_solar'] = MW_implemented['current-solar-production'].max()
        results.loc[(run, rs), 'MW_implemented_urban'] = MW_implemented['current-urban-production'].max()

### Save Results, as well as the original parameters

In [7]:
from datetime import datetime
dateTimeObj = datetime.now()

In [8]:
# Save Parameters
param_values_df.to_csv("results/parameters"+ str(dateTimeObj.year) + '-' + str(dateTimeObj.month) + '-' + str(dateTimeObj.day)+ "-" + str(dateTimeObj.hour) + '-' + str(dateTimeObj.minute) + "_"+str(no_samples)+" samples.csv")

# Save Results 
results.to_csv("results/outcomes"+ str(dateTimeObj.year) + '-' + str(dateTimeObj.month) + '-' + str(dateTimeObj.day)+ "-" + str(dateTimeObj.hour) + '-' + str(dateTimeObj.minute) +"_"+str(no_samples)+" samples.csv")

In [84]:
# to close the netlogo workspace
netlogo.kill_workspace()